In [1]:
from pulp import *
import pandas as pd
import numpy as np


# Inspired by https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0



In [41]:
# Data definition
# ---------------

number_of_warehouses = 2
number_of_customers = 4

# Cost Matrix - Rows: Warehouses, Columns: Customers
cost_matrix = np.array([[1, 3, 0.5, 4],
                        [2.5, 5, 1.5, 2.5]])

# Demand Vector - Rows: Customers
customer_demand = np.array([35000, 22000, 18000, 30000])

# Supply Vector - Rows: Warehouses
warehouse_supply = np.array([60000, 80000])


In [48]:
# Model Initialization

model = LpProblem(name="Supply-Demand_Problem", sense=LpMinimize)  # args: name, sense (minimize or maximize)

In [49]:
# Define Decision Variables

# Create a list of tuples containing all the possible combinations of
# indices for the warehouses and customers

variable_names = [str(i) + str(j) for j in range(1, number_of_customers + 1) for i in range(1, number_of_warehouses + 1)]
variable_names.sort()

print(f"Variable Indices: {variable_names}")



Variable Indices: ['11', '12', '13', '14', '21', '22', '23', '24']


In [50]:
# Define Variables - Rows: Warehouses, Columns: Customers

decision_variables = LpVariable.matrix(name="X", indices=variable_names, lowBound=0, cat="Integer")

allocation = np.array(decision_variables).reshape(number_of_warehouses, number_of_customers)

print(f"Decision Variables / Allocation Matrix: \n{allocation}")

Decision Variables / Allocation Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]


In [51]:
# Define Objective Function - Overall cost of supply
objective_function = lpSum(allocation * cost_matrix)  # sum-product of allocation and cost matrix

print(f"Objective Function: \n{objective_function}")

model += objective_function

print(f"Model: \n{model}")

Objective Function: 
X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24
Model: 
Supply-Demand_Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer



In [52]:
# Define Constraints - 2 types of constraints
# 1. Supply or Warehouse Constraints - Total supply from each warehouse must be less than or equal to the warehouse's supply
# 2. Demand or Customer Constraints - Total demand from each customer must be less than or equal to the customer's demand

# Supply or Warehouse Constraints

for i in range(number_of_warehouses):
    print(f"Supply Constraint Warehouse {i + 1}: {lpSum(allocation[i, :])} <= {warehouse_supply[i]}")
    model += lpSum(allocation[i, :]) <= warehouse_supply[i], f"Supply Constraint Warehouse {i + 1}"

Supply Constraint Warehouse 1: X_11 + X_12 + X_13 + X_14 <= 60000
Supply Constraint Warehouse 2: X_21 + X_22 + X_23 + X_24 <= 80000


In [53]:
# Customer or Demand Constraints

for j in range(number_of_customers):
    print(f"Demand Constraint Customer {j + 1}: {lpSum(allocation[:, j])} <= {customer_demand[j]}")
    model += lpSum(allocation[:, j]) >= customer_demand[j], f"Demand Constraint Customer {j + 1}"

Demand Constraint Customer 1: X_11 + X_21 <= 35000
Demand Constraint Customer 2: X_12 + X_22 <= 22000
Demand Constraint Customer 3: X_13 + X_23 <= 18000
Demand Constraint Customer 4: X_14 + X_24 <= 30000


In [37]:
# Check the model

print(f"Model: \n{model}")

Model: 
Supply-Demand_Problem:
MINIMIZE
1.0*x_11 + 3.0*x_12 + 0.5*x_13 + 4.0*x_14 + 2.5*x_21 + 5.0*x_22 + 1.5*x_23 + 2.5*x_24 + 0.0
SUBJECT TO
Supply_Constraint_Warehouse_1: x_11 + x_12 + x_13 + x_14 <= 60000

Supply_Constraint_Warehouse_2: x_21 + x_22 + x_23 + x_24 <= 80000

Demand_Constraint_Customer_1: x_11 + x_21 <= 35000

Demand_Constraint_Customer_2: x_12 + x_22 <= 22000

Demand_Constraint_Customer_3: x_13 + x_23 <= 18000

Demand_Constraint_Customer_4: x_14 + x_24 <= 30000

VARIABLES
0 <= x_11 Integer
0 <= x_12 Integer
0 <= x_13 Integer
0 <= x_14 Integer
0 <= x_21 Integer
0 <= x_22 Integer
0 <= x_23 Integer
0 <= x_24 Integer



In [54]:
# Save the model into a ".lp" file
model.writeLP("Supply-Demand_Problem.lp")

[X_11, X_12, X_13, X_14, X_21, X_22, X_23, X_24]

In [55]:
# Run the model and check the status


model.solve(PULP_CBC_CMD())
print(f"Status: {LpStatus[model.status]}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/frankkelly/Library/Caches/pypoetry/virtualenvs/linear-programming-sahaj-3qyTN_YL-py3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/17/sgfdr00n1sn9ddjf831jrsd00000gn/T/5ac7dbd7bd134e898558009ff45352ae-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/17/sgfdr00n1sn9ddjf831jrsd00000gn/T/5ac7dbd7bd134e898558009ff45352ae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 68 ENDATA
Problem MODEL has 6 rows, 8 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 200000 - 0.00 seconds
Cgl0004I processed model has 6 rows, 8 columns (8 integer (0 of which binary)) and 16 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of 200000 found by DiveCoeffi

In [56]:
# Print the results (Objective Function Value and Decision Variables Values)

print(f"Total Cost: {value(model.objective)}")

# Print the decision variables values

for variable in model.variables():
    try: 
        print(f"{variable.name}: {variable.varValue}")
    except AttributeError:
        print(f"{variable.name}: {variable.value()}")

        


Total Cost: 200000.0
X_11: 35000.0
X_12: 22000.0
X_13: 3000.0
X_14: 0.0
X_21: 0.0
X_22: 0.0
X_23: 15000.0
X_24: 30000.0


In [60]:
# Warehouse 1 and 2 required capacity

for i in range(number_of_warehouses):
    print(f"Required Capacity Warehouse {i + 1}: {lpSum(allocation[i][j].value() for j in range(number_of_customers))}")


Required Capacity Warehouse 1: 60000.0
Required Capacity Warehouse 2: 45000.0
